In [231]:
import pandas as pd
import numpy as np

In [232]:
df = pd.read_csv('C:/Users/ScMa9004/Downloads/RULES_1579101873184.csv', sep = ',', low_memory=False)

In [233]:
# из выгрузки правил вырезать нужные
df = df[df['CREATED_BY'] == 'maxim.schenin@nielsen.com']

In [234]:
res = pd.DataFrame(columns = {'CELL_AREA'})

In [235]:
arc = pd.read_excel('C:/Users/ScMa9004/Documents/All Russia cities 01.01.2019_ for_2020.xlsx',
                    sheet_name = 'All_Russia_cities', skiprows = 1)
#replase MSC to 1
arc.loc[arc['RMS_CLUSTER_CODE (справочник)'].isin(['MSC1', 'MSC2']), 'RMS_CLUSTER_CODE (справочник)'] = 1 

In [236]:
uo200 = pd.read_excel('C:/Users/ScMa9004/Documents/Universe Over 200.xls', sheet_name = 'Towns&Villages', skiprows = 1)
#drop nonaudited subjects
uo200 = uo200[~uo200['Federal Subject Number'].isin([4, 42, 43, 44, 45, 46, 47, 48, 60, 68, 70, 71, 79, 80, 81, 85, 87, 88])]
#replase MSC to 1
uo200.loc[uo200['RMS_CLUSTER_CODE (ACR)'].isin(['MSC1', 'MSC2']), 'RMS_CLUSTER_CODE (ACR)'] = 1

In [237]:
rules = pd.read_excel('C:/Users/ScMa9004/Documents/Panel/NEW PANEL/FMCG_CIG_BEER.xlsx', 
                                  sheet_name = 'index + mcellname')

In [238]:
def parser(mcell):
    if mcell.split('-')[0] in ['CNT', 'FST', 'VOL', 'URL', 'NWT', 'STH', 'MSO', 'SIB']:
        area = [mcell.split('-')[0]]
        if 'RR' in mcell.split('-')[-1]:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            types = ['OSU|OLA|OME|OSM|OPV|OMM|OMS']
            pgr = list(mcell.split('-')[2].split('|'))
            ds = list(mcell.split('-')[3].split('|'))
            hip = []
            ndx = [mcell.split('-')[-1]]
            NO_city = []
            ttr = ['MT|TT']
        else:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
                types = ['OKS', 'ONS', 'OPV', 'OSM']
            else:
                types = list(mcell.split('-')[2].split('|'))
            pgr = list(map(int, mcell.split('-')[3].split('|')))
            ttr = list(mcell.split('-')[4].split('|'))
            try:
                NO_city = list(map(int, mcell.split('-')[5].replace('(', '').replace(')', '').split('&')))
            except:
                NO_city = []
            hip = []
            ndx = [mcell.split('-')[-1]]
            ds = []
    else:
        if '|' in mcell.split('-')[0]:
            area = [mcell.split('-')[0] + ' ' + mcell.split('-')[1]]
        else:
            area = [mcell.split('-')[1]]
        ccodes = list(map(int, mcell.split('-')[0].split('|')))
        subreg = []
        types = list(mcell.split('-')[2].split('|'))
        if ('Y' in mcell.split('-')[3]) | ('N' in mcell.split('-')[3]):
            ttr = ['TT']
            hip = list(mcell.split('-')[3])
        else:
            ttr = list(mcell.split('-')[3].split('|'))
            hip = []
        NO_city = []
        pgr = []
        ndx = [mcell.split('-')[-1]]
        ds = []
    return(area, ccodes, subreg, types, pgr, ttr, NO_city, hip, ds, ndx)

# CELL_AREA MAPPING

In [239]:
for i in df['RULE_ID'][df['RULE_TYPE'] == 'MICRO_CELL'].unique():
    try:
        a = df['RULE_ID'][(df['RULE_TYPE'] == 'CELL_AREA') &
            (df['ATTRIBUTE_ID'] == 1001023) &
              (df['VALUE'] == df['VALUE'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 1001023)].iloc[0]) &
              (df['OPERATOR'] == df['OPERATOR'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 1001023)].iloc[0])]
    except:
        a = df['RULE_ID'][df['RULE_TYPE'] == 'CELL_AREA'].unique()
    try:
        b = df['RULE_ID'][(df['RULE_TYPE'] == 'CELL_AREA') &
                    (df['ATTRIBUTE_ID'] == 1001022) &      
              (df['VALUE'] == df['VALUE'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 1001022)].iloc[0]) &
              (df['OPERATOR'] == df['OPERATOR'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 1001022)].iloc[0])]
    except:
        b = df['RULE_ID'][df['RULE_TYPE'] == 'CELL_AREA'].unique()
    try:
        c = df['RULE_ID'][(df['RULE_TYPE'] == 'CELL_AREA') &
                        (df['ATTRIBUTE_ID'] == 20125) &  
              (df['VALUE'] == df['VALUE'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 20125)].iloc[0]) &
              (df['OPERATOR'] == df['OPERATOR'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 20125)].iloc[0])]
    except:
        c = df['RULE_ID'][df['RULE_TYPE'] == 'CELL_AREA'].unique()
    try:
        d = df['RULE_ID'][(df['RULE_TYPE'] == 'CELL_AREA') &
                          (df['ATTRIBUTE_ID'] == 21118) &
              (df['VALUE'] == df['VALUE'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 21118)].iloc[0]) &
              (df['OPERATOR'] == df['OPERATOR'][(df['RULE_ID'] == i) & (df['ATTRIBUTE_ID'] == 21118)].iloc[0])]
    except:
        d = df['RULE_ID'][df['RULE_TYPE'] == 'CELL_AREA'].unique()
        
    e = set(a) & set(b) & set(c) & set(d)
    
    res.loc[i, 'CELL_AREA'] = e
    
res = res.reset_index(drop = False)
    
res['CELL_AREA'] = res['CELL_AREA'].astype('str')
    
res['CELL_AREA'] = res['CELL_AREA'].str.replace('{', '')
res['CELL_AREA'] = res['CELL_AREA'].str.replace('}', '')

for i, j in res.iterrows():
    if len(j['CELL_AREA'].split(',')) > 1:
        if len(df[df['RULE_ID'] == int(j['CELL_AREA'].split(',')[0])]) ==\
        len(df[(df['RULE_ID'] == j['index']) &\
                (df['ATTRIBUTE_NAME'].isin(df['ATTRIBUTE_NAME'][df['RULE_ID'] == int(j['CELL_AREA'].split(',')[0])]))]):
            res.loc[i, 'CELL_AREA'] = int(j['CELL_AREA'].split(',')[0])
        else:
            res.loc[i, 'CELL_AREA'] = int(j['CELL_AREA'].split(',')[1])
            
res = res.astype('int')

In [240]:
res = pd.merge(res.rename(columns = {'index': 'RULE_ID'}), df[['RULE_ID', 'RULE_NAME']])

In [241]:
res['ndx'] = np.nan

for i, j in res.iterrows():
    res.loc[i, 'ndx'] = j['RULE_NAME'].split('-')[-1]

res.loc[res['ndx'] == 'FMCG', 'INDEX_ID'] = 1000296
res.loc[(res['ndx'] == 'CIG') | (res['ndx'] == 'RRC'), 'INDEX_ID'] = 1000334
res.loc[(res['ndx'] == 'BEER') | (res['ndx'] == 'RRB'), 'INDEX_ID'] = 1000328
res.loc[res['ndx'] == 'EXXON', 'INDEX_ID'] = 1001395

res.loc[res['ndx'] == 'FMCG', 'INDEX_NAME'] = 'RU FMCG AUDIT - DUPLICATE'
res.loc[(res['ndx'] == 'CIG') | (res['ndx'] == 'RRC'), 'INDEX_NAME'] = 'RU CIG AUDIT - DUPLICATE'
res.loc[(res['ndx'] == 'BEER') | (res['ndx'] == 'RRB'), 'INDEX_NAME'] = 'RU BEER AUDIT - DUPLICATE'
res.loc[res['ndx'] == 'EXXON', 'INDEX_NAME'] = 'RU EXXON AUDIT'

In [242]:
res = res.drop_duplicates()

# URBAN split by cities

In [243]:
def parser_2(mcell):
    if mcell.split('-')[0] in ['CNT', 'FST', 'VOL', 'URL', 'NWT', 'STH', 'MSO', 'SIB']:
        area = [mcell.split('-')[0]]
        if 'RR' in mcell.split('-')[-1]:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            types = ['OSU|OLA|OME|OSM|OPV|OMM|OMS']
            pgr = list(mcell.split('-')[2].split('|'))
            ds = list(mcell.split('-')[3].split('|'))
            hip = []
            ndx = [mcell.split('-')[-1]]
            NO_city = []
            ttr = ['MT|TT']
        else:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            #if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
            #    types = ['OKS', 'ONS', 'OPV', 'OSM']
            #else:
            types = list(mcell.split('-')[2].split('|'))
            pgr = list(map(int, mcell.split('-')[3].split('|')))
            ttr = list(mcell.split('-')[4].split('|'))
            try:
                NO_city = list(map(int, mcell.split('-')[5].replace('(', '').replace(')', '').split('&')))
            except:
                NO_city = []
            hip = []
            ndx = [mcell.split('-')[-1]]
            ds = []
    else:
        if '|' in mcell.split('-')[0]:
            area = [mcell.split('-')[0] + ' ' + mcell.split('-')[1]]
        else:
            area = [mcell.split('-')[1]]
        ccodes = list(map(int, mcell.split('-')[0].split('|')))
        subreg = []
        types = list(mcell.split('-')[2].split('|'))
        if ('Y' in mcell.split('-')[3]) | ('N' in mcell.split('-')[3]):
            ttr = ['TT']
            hip = list(mcell.split('-')[3])
        else:
            ttr = list(mcell.split('-')[3].split('|'))
            hip = []
        NO_city = []
        pgr = []
        ndx = [mcell.split('-')[-1]]
        ds = []
    return(area, ccodes, subreg, types, pgr, ttr, NO_city, hip, ds, ndx)

In [244]:
#prepare file
def urban_syndi(df, arc):
    
    # some types should stay together
    for i, j in df.iterrows():
        df.loc[i, 'new_mcell_name'] = df['new_mcell_name'][i].replace('FKF|FKT|FKM|FKC', 'FKF/FKT/FKM/FKC')
    
    
    res = pd.DataFrame(columns = {'city', 'FMCG', 'CIG', 'BEER', 'EXXON', 'type', 'ttr', 'MCELL_ID'})
    
    n = 0
    for i, j in df[df['index'] < 10000].iterrows():
        wrk = parser_2(j['new_mcell_name'])
        if len(wrk[1]) > 0:
            cities = wrk[1]
        else:
            cities = arc['City Code'][arc['# of Federal Subject'].isin(wrk[2]) & 
                                      arc['pgr1'].isin(wrk[4]) & 
                                      ~arc['City Code'].isin(wrk[6]) &
                                      arc['City Code'].notna() &
                                      arc['Restrict for audit '].isin([0])]
        for k in cities:
            for l in wrk[3]:
                for m in wrk[5]:
                    res.loc[n, 'city'] = k
                    res.loc[n, 'type'] = l
                    res.loc[n, 'ttr'] = m
                    if 'FMCG' in wrk[9]:
                        res.loc[n, 'FMCG'] = df['MCELL_ID'][i]
                    if 'CIG' in wrk[9]:
                        res.loc[n, 'CIG'] = df['MCELL_ID'][i]
                    if 'BEER' in wrk[9]:
                        res.loc[n, 'BEER'] = df['MCELL_ID'][i]
                    if 'EXXON' in wrk[9]:
                        res.loc[n, 'EXXON'] = df['MCELL_ID'][i]
                    n += 1 
    
    res = res.fillna(0)
    
    res = pd.merge(res, 
                   arc[['City Code', 'RMS_CLUSTER_CODE (справочник)']].rename(columns =\
                                                {'City Code': 'city', 'RMS_CLUSTER_CODE (справочник)': 'ops'}).dropna(), 
                   on = 'city', 
                   how = 'left')
    
    pivot = pd.pivot_table(res, 
                           index = ['ops', 'city', 'type', 'ttr'], 
                           values = ['FMCG', 'CIG', 'BEER', 'EXXON'], 
                           aggfunc = np.sum).reset_index()
    
    pivot['city'] = pivot['city'].astype('int')
    
    pivot = pd.pivot_table(pivot, 
                           index = ['ops', 'type', 'ttr', 'FMCG', 'CIG', 'BEER', 'EXXON'], 
                           values = ['city'], 
                           aggfunc = lambda x: list(x.unique())).reset_index()
    
    pivot['city'] = pivot['city'].astype('str')
    
    pivot = pd.pivot_table(pivot,
               index = ['ops', 'ttr', 'FMCG', 'CIG', 'BEER', 'EXXON', 'city'], 
               values = 'type', 
               aggfunc = lambda x: list(x.unique())).reset_index()
    
    pivot['type'] = pivot['type'].astype('str')
    
    pivot = pd.pivot_table(pivot,
               index = ['ops', 'type', 'FMCG', 'CIG', 'BEER', 'EXXON', 'city'], 
               values = 'ttr', 
               aggfunc = lambda x: list(x.unique())).reset_index()
    
    pivot['ttr'] = pivot['ttr'].astype('str')
    
    pivot.loc[pivot['ops'] == 'MSC2', 'ops'] = 1
    
    return pivot

In [245]:
pivot = urban_syndi(rules, arc)

# RURAL syndicated split

In [246]:
def parser_1(mcell):
    if mcell.split('-')[0] in ['CNT', 'FST', 'VOL', 'URL', 'NWT', 'STH', 'MSO', 'SIB']:
        area = [mcell.split('-')[0]]
        if 'RR' in mcell.split('-')[-1]:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            types = ['OSU', 'OLA', 'OME', 'OSM', 'OPV', 'OMM', 'OMS']
            pgr = list(mcell.split('-')[2].split('|'))
            ds = list(mcell.split('-')[3].split('|'))
            hip = []
            ndx = [mcell.split('-')[-1]]
            NO_city = []
            ttr = ['MT|TT']
        else:
            ccodes = []
            subreg = list(map(int, mcell.split('-')[1].split('|')))
            #if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
            #    types = ['OKS', 'ONS', 'OPV', 'OSM']
            #    hip = [mcell.split('-')[2]]
            #else:
            #next 2 rows
            types = list(mcell.split('-')[2].split('|'))
            hip = []
            pgr = list(map(int, mcell.split('-')[3].split('|')))
            ttr = list(mcell.split('-')[4].split('|'))
            try:
                NO_city = list(map(int, mcell.split('-')[5].replace('(', '').replace(')', '').split('&')))
            except:
                NO_city = []
            ndx = [mcell.split('-')[-1]]
            ds = []
    else:
        if '|' in mcell.split('-')[0]:
            area = [mcell.split('-')[0] + ' ' + mcell.split('-')[1]]
        else:
            area = [mcell.split('-')[1]]
        ccodes = list(map(int, mcell.split('-')[0].split('|')))
        subreg = []
        hip = []
        #if ('IML' in mcell.split('-')[2]) | ('IMS' in mcell.split('-')[2]):
        #    types = ['OKS', 'ONS', 'OPV', 'OSM']
        #    hip = [mcell.split('-')[2]]
        #else:
        types = list(mcell.split('-')[2].split('|'))
        if ('Y' in mcell.split('-')[3]) | ('N' in mcell.split('-')[3]):
            ttr = ['TT']
            hip = list(mcell.split('-')[3])
        else:
            ttr = list(mcell.split('-')[3].split('|'))
        NO_city = []
        pgr = []
        ndx = [mcell.split('-')[-1]]
        ds = []
    return(area, ccodes, subreg, types, pgr, ttr, NO_city, hip, ds, ndx)

In [247]:
#prepare file
def rural_syndi(df):
    
    res = pd.DataFrame(columns = {'sub', 'RRC', 'RRB', 'pgr', 'DS', 'MCELL_ID'})
    
    #split
    n = 0
    for i, j in df[df['index'] >= 10000].iterrows():
        wrk = parser_1(j['new_mcell_name'])
        for k in wrk[2]:
            for l in wrk[4]:
                for m in wrk[8]:
                    res.loc[n, 'sub'] = k
                    res.loc[n, 'pgr'] = l
                    res.loc[n, 'DS'] = m
                    res.loc[n, 'MCELL_ID'] = df['MCELL_ID'][i]
                    if 'RRC' in wrk[9]:
                        res.loc[n, 'RRC'] = j['index']
                    if 'RRB' in wrk[9]:
                        res.loc[n, 'RRB'] = j['index']
                    n += 1 
                    
    #syndi
    for i, j in res[res['RRC'].notna()].iterrows():
        try:
            res.loc[i, 'RRB'] = res['MCELL_ID'][(res['pgr'] == j['pgr']) &
                                           (res['DS'] == j['DS']) &
                                           (res['sub'] == j['sub']) & res.RRB.notna()].iloc[0]
            
            res = res.drop([res[(res['pgr'] == j['pgr']) &
                                (res['DS'] == j['DS']) &
                                (res['sub'] == j['sub']) & 
                                res.RRB.notna() & 
                                res.RRC.isna()].index.item()])
        except:
            pass
    
    #szhatie
    res = res.fillna(0)
    res['sub'] = res['sub'].astype('str')
    res = pd.pivot_table(res, 
                         index = ['RRC', 'RRB'],
                         values = ['sub', 'pgr', 'DS', 'MCELL_ID'], 
                         aggfunc = lambda x: x.unique()).reset_index()
    
    #replace remains of indexes on mcell ids
    res.loc[(res['RRC'] != 0) & (res['RRB'] != 0), 'RRC'] = res['MCELL_ID']
    res.loc[res['RRC'] == 0, 'RRB'] = res['MCELL_ID']
    res.loc[res['RRB'] == 0, 'RRC'] = res['MCELL_ID']
    
    res = res.drop(columns = 'MCELL_ID')
     
    return res

In [248]:
rrl = rural_syndi(rules)

# Replace

In [249]:
# separate RRL
rural = res[res['ndx'].isin(['RRB', 'RRC'])]
urban = res[~res['ndx'].isin(['RRB', 'RRC'])]

In [250]:
# urban
fmcg = pivot.rename(columns = {'FMCG' : 'RULE_ID'}).drop(columns = {'CIG', 'BEER', 'EXXON'})
cig = pivot.rename(columns = {'CIG' : 'RULE_ID'}).drop(columns = {'FMCG', 'BEER', 'EXXON'})
beer = pivot.rename(columns = {'BEER' : 'RULE_ID'}).drop(columns = {'CIG', 'FMCG', 'EXXON'})
exxon = pivot.rename(columns = {'EXXON' : 'RULE_ID'}).drop(columns = {'CIG', 'BEER', 'FMCG'})

urbn = fmcg.append(cig).append(beer).append(exxon)

urbn = urbn[urbn.RULE_ID != 0]

urbn = urbn.reset_index(drop = True)

urbn = pd.merge(urbn, urban, on = 'RULE_ID', how = 'left').dropna()

urbn['type'] = urbn['type'].str.replace(']', '')
urbn['type'] = urbn['type'].str.replace('[', '')
urbn['type'] = urbn['type'].str.replace(', ', '|')
urbn['type'] = urbn['type'].str.replace("'", "")

urbn['ttr'] = urbn['ttr'].str.replace(']', '')
urbn['ttr'] = urbn['ttr'].str.replace('[', '')
urbn['ttr'] = urbn['ttr'].str.replace(', ', '|')
urbn['ttr'] = urbn['ttr'].str.replace("'", "")

urbn['city'] = urbn['city'].str.replace(']', '')
urbn['city'] = urbn['city'].str.replace('[', '')
urbn['city'] = urbn['city'].str.replace(', ', '|')
urbn['city'] = urbn['city'].str.replace("'", "")


urbn['SAMPLE_AREA_NAME'] = urbn['ops'].astype('int').astype('str') + ' ' + urbn['type'] + '-' + urbn['ttr'] + '-' + urbn['city']
urbn['REGION_NAME'] = urbn['SAMPLE_AREA_NAME']


#k = 0
#for i in urbn['REGION_NAME'].unique():
#    k += 1
#    urbn.loc[urbn['REGION_NAME'] == i, 'REGION_ID'] = k
    
#k = 0
#for i in urbn['SAMPLE_AREA_NAME'].unique():
#    k += 1
#    urbn.loc[urbn['SAMPLE_AREA_NAME'] == i, 'SAMPLE_AREA_ID'] = k
    
urbn['REGION_ID'] = 0
urbn['SAMPLE_AREA_ID'] = 0
    
urbn = urbn[['INDEX_ID', 'INDEX_NAME', 'REGION_ID', 'REGION_NAME', 'SAMPLE_AREA_ID',
           'SAMPLE_AREA_NAME', 'CELL_AREA', 'RULE_ID']].rename(columns = {'CELL_AREA': 'CELL_AREA_RULE_ID',
                                                                          'RULE_ID': 'MICRO_CELL_RULE_ID'})

urbn['EXCLUDE_FLAG'] = 'N'
urbn['EXCLUSION_REASON'] = 'N'

urbn['INDEX_ID'] = urbn['INDEX_ID'].astype('int')
urbn['REGION_ID'] = urbn['REGION_ID'].astype('int')
urbn['SAMPLE_AREA_ID'] = urbn['SAMPLE_AREA_ID'].astype('int')
urbn['CELL_AREA_RULE_ID'] = urbn['CELL_AREA_RULE_ID'].astype('int')

In [251]:
# rural
rrc = rrl
rrb = rrl

rrc = rrc.rename(columns = {'RRC': 'RULE_ID'}).drop(columns = 'RRB')
rrb = rrb.rename(columns = {'RRB': 'RULE_ID'}).drop(columns = 'RRC')

rrl = rrc.append(rrb)

rrl = rrl[rrl.RULE_ID != 0]

rrl = rrl.reset_index(drop = True)

rrl = pd.merge(rrl, rural, on = 'RULE_ID', how = 'left')


for i, j in rrl.iterrows():
    if ',' in str(j['sub']):
        subreg = str(j['sub']).replace('[', '').replace(']', '').replace(',', '|').replace(' ', '').replace("'", "")
    else:
        subreg = str(j['sub']).replace('[', '').replace(']', '').replace(' ', '|').replace("'", "")
    rrl.loc[i, 'RULE_NAME'] = j['RULE_NAME'].split('-')[0] + '-' + subreg + '-' + j['pgr'] + '-' + j['DS'] + '-RRL'
    
rrl = rrl[['RULE_ID', 'CELL_AREA', 'RULE_NAME', 'ndx', 'INDEX_ID', 'INDEX_NAME']]

In [252]:
# rural x OPS

fin = rrl[0:0]

for i, j in rrl.iterrows():
    wrk = parser(j['RULE_NAME'])
    subreg = uo200['Federal Subject Number'].isin(wrk[2])
    pgr = uo200['pgr_MT'].isin(wrk[4])
    dist = uo200['Distance Strata'].isin(wrk[8])
    for k in uo200['RMS_CLUSTER_CODE (ACR)'][subreg & pgr & dist].unique():
        j['cluster'] = k
        fin = fin.append(j)

fin = fin.dropna()
fin = fin.reset_index(drop = True)


fin['SAMPLE_AREA_NAME'] = fin['cluster'].astype('int').astype('str') + ' ' + fin['RULE_NAME']
#fin['SAMPLE_AREA_NAME'] = fin['SAMPLE_AREA_NAME'].str.replace('.0', '')

temp = df[['RULE_ID', 'RULE_NAME']][df['RULE_TYPE'] == 'CELL_AREA'].\
        rename(columns = {'RULE_ID': 'CELL_AREA', 'RULE_NAME': 'REGION_NAME'}).drop_duplicates()

fin = pd.merge(fin, temp, on = 'CELL_AREA', how = 'inner')

#k = 0
#for i in fin['REGION_NAME'].unique():
#    k += 1
#    fin.loc[fin['REGION_NAME'] == i, 'REGION_ID'] = k
    
#k = 0
#for i in fin['SAMPLE_AREA_NAME'].unique():
#    k += 1
#    fin.loc[fin['SAMPLE_AREA_NAME'] == i, 'SAMPLE_AREA_ID'] = k

fin['REGION_ID'] = 0
fin['SAMPLE_AREA_ID'] = 0


fin = fin.rename(columns = {'CELL_AREA': 'CELL_AREA_RULE_ID', 'RULE_ID': 'MICRO_CELL_RULE_ID'})

fin = fin[['INDEX_ID', 'INDEX_NAME', 'REGION_ID', 'REGION_NAME', 'SAMPLE_AREA_ID',
           'SAMPLE_AREA_NAME', 'CELL_AREA_RULE_ID', 'MICRO_CELL_RULE_ID']]

fin = fin.drop_duplicates()

fin['EXCLUDE_FLAG'] = 'N'
fin['EXCLUSION_REASON'] = 'N'

fin['INDEX_ID'] = fin['INDEX_ID'].astype('int')
fin['REGION_ID'] = fin['REGION_ID'].astype('int')
fin['SAMPLE_AREA_ID'] = fin['SAMPLE_AREA_ID'].astype('int')
fin['CELL_AREA_RULE_ID'] = fin['CELL_AREA_RULE_ID'].astype('int')

In [253]:
mapping = urbn.append(fin)

In [254]:
k = 0
for i in mapping['REGION_NAME'].unique():
    k += 1
    mapping.loc[mapping['REGION_NAME'] == i, 'REGION_ID'] = k
    
k = 0
for i in mapping['SAMPLE_AREA_NAME'].unique():
    k += 1
    mapping.loc[mapping['SAMPLE_AREA_NAME'] == i, 'SAMPLE_AREA_ID'] = k

In [255]:
mapping.to_csv('mapping_new-panel_full_splitted.csv', sep = ',', index = False)